In [4]:
import numpy as np
from scipy.integrate import solve_ivp
import sk_dsp_comm.sigsys as ss

def sigmoid(u):

    return np.tanh(u)

def two_oscillators(t, y, h_ex, h_in, pars, frac_E, frac_I, sr, time_stop, pert):
               
    tau_ex, tau_in, c2, c4, c_EE, c_EI = pars

    time_index = int(t*sr)

    if time_index >= time_stop*sr:

        dydt = np.zeros(4)

        return dydt


    else:
        
        dydt = (

            (pert[time_index] - y[0] - c2*sigmoid(y[1]) + c_EE*sigmoid(y[0]+frac_E*y[2]))*tau_ex,
             
            (h_in             - y[1] - c4*sigmoid(y[1]) + c_EI*sigmoid(y[0]+frac_I*y[2]))*tau_in,

             
            (h_ex             - y[2] - c2*sigmoid(y[3]) + c_EE*sigmoid(y[2]+frac_E*y[0]))*tau_ex,
            
            (h_in             - y[3] - c4*sigmoid(y[3]) + c_EI*sigmoid(y[2]+frac_I*y[0]))*tau_in
           )

    return dydt

# Input parameter
h_ex_0 = -7.0
h_in_0 = -4.0

# Oscillator parameters
pars = (1, 2.5, 10, 0, 5, 10) # Homoclinic

# Oscillator coupling
frac_E, frac_I = 0.2, 0. # 0: no coupling

# Initial conditions
y_ini = [ -1.9, -13.56, -1.9, -13.56]

# Time array
time_stop = 150
sr        = 1000
time      = np.linspace(start=0, stop=time_stop, num=time_stop*sr)

rows = time.size

pulse_wid = 4.
pulse_amp = 0.
pulse_per = 10

pert = h_ex_0 + pulse_amp*ss.rect(np.mod(time, pulse_per)-(pulse_per)/2+pulse_wid/2, pulse_wid)


# Simulation
solution = solve_ivp(fun=two_oscillators, t_span=(0, time_stop), y0=y_ini, t_eval=time,
              args=(h_ex_0, h_in_0, pars, frac_E, frac_I, sr, time_stop, pert), 
              method='BDF', max_step=0.1)

t, y_pert = solution.t, solution.y.T


150000